In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA Minimization

In this notebook, we verify our minimization algorithm using a concrete example.
We manually construct a DFA, visualize it, run the minimization, and visualize the result.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from "recursive-set";


import { minimize, DFA, TransRelDet, DFAState, State } from "./07-Minimize";
import { dfa2string,dfa2dot } from "./FSM-2-Dot";
import { key } from "./05-DFA-2-RegExp";

// Initialize Graphviz
const viz = await instance();

### Defining the DFA

We define the DFA $A = (Q, \Sigma, \delta, q_0, F)$ manually.
To match our Type System (`DFAState = RecursiveSet<State>`), we use a helper `S(i)` to wrap the state IDs (0-8) into Sets.

**Transitions:**
* (0, a) $\to$ 1, (0, b) $\to$ 6
* ... (and so on)

In [ ]:
// 1. Pre-create ALL state instances (Singletons {0}..{8})
// RecursiveSet.fromSortedUnsafe is the fastest way to create singletons in v7
const states: DFAState[] = Array.from({ length: 9 }, (_, i) => 
    RecursiveSet.fromSortedUnsafe([i])
);

// 2. Helper: Access the cached instance by index
const S = (i: number) => states[i];

// The alphabet
const Sigma = new RecursiveSet<string>("a", "b");

// The Transition Function
const delta = new Map<string, DFAState>();

const addTrans = (from: number, char: string, to: number) => {
  delta.set(key(S(from), char), S(to));
};

// Define transitions
addTrans(0, "a", 1); addTrans(0, "b", 6);
addTrans(1, "a", 2); addTrans(1, "b", 7);
addTrans(2, "a", 3); addTrans(2, "b", 6);
addTrans(3, "a", 0); addTrans(3, "b", 7);
addTrans(4, "a", 5); addTrans(4, "b", 2);
addTrans(5, "a", 6); addTrans(5, "b", 8);
addTrans(6, "a", 7); addTrans(6, "b", 2);
addTrans(7, "a", 0); addTrans(7, "b", 3);
addTrans(8, "a", 4); addTrans(8, "b", 8);

// Set of all states Q: Efficient Bulk Creation
const Q = RecursiveSet.fromArray(states);

// Accepting States A = {2, 3, 6, 7}
const A_states = RecursiveSet.fromArray([S(2), S(3), S(6), S(7)]);

// The Start State q0
const q0 = S(0);

// Construct the DFA object
const A: DFA = {
  Q: Q,
  Sigma: Sigma,
  delta: delta,
  q0: q0,
  A: A_states
};

### Visualize Original DFA

We generate both the text representation (to see the state encoding) and the Graphviz diagram.

**Note:** The state names (S0, S1...) in the graph are generated automatically. Check the "state encoding" in the text output to map them back to our sets (e.g., see which S-name corresponds to `{0}`).

In [ ]:
console.log(dfa2string(A));

const { dot: dot1 } = dfa2dot(A);
display.html(viz.renderString(dot1, { format: "svg" }));

## Minimize

We apply the `minimize` function.

**Note:** Our optimized implementation in `07-Minimize.ts` automatically normalizes the resulting equivalence classes. This means the result `F` is a standard `DFA` where states are simple identifiers (e.g., `{0}`, `{1}`), not nested sets of sets.

In [ ]:
const F : DFA = minimize(A);

### Visualize Minimized DFA

Now we can use our standard tools without extra casting.

The output shows the merged states (e.g., `{{3}, {7}}`).

In [ ]:
console.log("Minimized DFA Structure:");
console.log(dfa2string(F));

const { dot: dot2 } = dfa2dot(F);
display.html(viz.renderString(dot2, { format: "svg" }));